In [3]:
from dataset import TensorflowDataset
import tensorflow as tf
import numpy as np
import pandas as pd
import csv
# 

c:\Users\mikol\AppData\Local\Programs\Python\Python310\lib\site-packages\tqdm\auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [4]:
test_data = pd.read_pickle('extracted_features\\features_test.pkl')
test = tf.data.Dataset.from_tensor_slices((test_data, np.zeros(len(test_data))))

In [22]:
# test_silence_data = pd.read_pickle('extracted_features\\features_test_silence.pkl')
# test_silence = tf.data.Dataset.from_tensor_slices((test_silence_data, np.zeros(len(test_silence_data))))

# silence + unknown + labels model

In [28]:
import numpy as np
import models
from models import Lstm, Gru, Transformer, TransformerBlock
from dataset import LABELS
import pandas as pd

class final_model_SKL:
    def __init__(self, name, test, batch_size=32):

        try:
            model_known = tf.keras.models.load_model(f'models\\final\\TEST_{name}_known.h5')
            model_label = tf.keras.models.load_model(f'models\\final\\TEST_{name}_labels_only.h5')
        except:
            model_known = tf.keras.models.load_model(f'models\\final\\TEST_{name}_known.h5',custom_objects={"TransformerBlock": TransformerBlock})
            model_label = tf.keras.models.load_model(f'models\\final\\TEST_{name}_labels_only.h5',custom_objects={"TransformerBlock": TransformerBlock})
            
        self.model_known = model_known
        self.model_label = model_label
        self.test = test
        self.batch_size = batch_size
        self.name = name

        LABELS_dict = {i:LABELS[i] for i in range(len(LABELS))}
        LABELS_dict[10] = 'unknown'
        self.LABELS_dict = LABELS_dict

        self.predicted_labels = None
        self.silence_labels = None

    def predict_silence(self):
        self.silence_labels = pd.read_pickle('silence_detected.pkl')
    
    def predict_label(self):
        self.predicted_labels = self.model_label.predict(self.test.batch(self.batch_size))

    def predict_if_known(self):
        self.predicted_labels_known = self.model_known.predict(self.test.batch(self.batch_size))

    def save_results(self):
        df = pd.read_csv('sample_submission.csv')
        x = pd.concat([pd.DataFrame(np.argmax(d.predicted_labels, axis=1)),pd.DataFrame(d.silence_labels), pd.DataFrame(np.argmax(d.predicted_labels_known, axis=1))], axis=1)
        x.columns = ['label', 'silence','unknown']
        # unknown == 1 => unknown
        res = np.where(x['silence'] == 0 , 'silence', np.where((x['unknown'] == 1) , 'unknown', [d.LABELS_dict[i] for i in x['label']]))
        df['label'] = res
        df.to_csv(f'results/FINAL_submission_SKL_{self.name}.csv', index=False, header=True)

In [32]:
d = final_model_SKL('gru', test)

In [33]:
d.predict_if_known()
d.predict_label()
d.predict_silence()

4955/4955 [==============================] - 70s 14ms/step


In [34]:
d.save_results()

In [25]:
df = pd.read_csv('sample_submission.csv')
x = pd.concat([pd.DataFrame(np.argmax(d.predicted_labels, axis=1)),pd.DataFrame(d.silence_labels), pd.DataFrame(np.argmax(d.predicted_labels_known, axis=1))], axis=1)
x.columns = ['label', 'silence','unknown']
# unknown == 1 => unknown
res = np.where(x['silence'] == 0 , 'silence', np.where((x['unknown'] == 1) , 'unknown', [d.LABELS_dict[i] for i in x['label']]))
df['label'] = res
name = 'transformer'
df.to_csv(f'results/FINAL_submission_SKL_{name}.csv', index=False, header=True)

# SILENCE - labels/unknown HIPERPARAMS

In [61]:
gru = pd.read_pickle('results/gru_wyniki_.pkl')

In [62]:
lstm = pd.read_pickle('results/lstm_wyniki_.pkl')
transformer = pd.read_pickle('results/transformer_wyniki_.pkl')

In [65]:
print('Best parameter for LSTM model to predict known labels and "unknown":')
lstm[lstm.val_accuracy_max == lstm.val_accuracy_max.max()]

Best parameter for LSTM model to predict known labels and "unknown":


,lstm_units,dropout_rate,learning_rate,epoch,batch,loss_max,accuracy_max,val_loss_max,val_accuracy_max
18,128.0,0.2,0.001,20.0,32.0,0.122819,0.960497,1.240723,0.752133


In [64]:
print('Best parameter for GRU model to predict known labels and "unknown":')
gru[gru.val_accuracy_max == gru.val_accuracy_max.max()]

Best parameter for GRU model to predict known labels and "unknown":


,gru_units,dropout_rate,learning_rate,epoch,batch,loss_max,accuracy_max,val_loss_max,val_accuracy_max
10,64.0,0.2,0.001,30.0,64.0,0.367678,0.881785,0.63184,0.820683


In [66]:
print('Best parameter for Transformer model to predict known labels and "unknown":')
transformer[transformer.val_accuracy_max == transformer.val_accuracy_max.max()]

Best parameter for Transformer model to predict known labels and "unknown":


,num_heads,dropout_rate,learning_rate,epoch,batch,num_layer,loss_max,accuracy_max,val_loss_max,val_accuracy_max
0,2.0,0.2,0.001,20.0,64.0,2.0,0.524436,0.830724,0.692129,0.800677


# Silence +unknown + label HIPERPARAMS

In [47]:
df_kn_lstm = pd.read_pickle('results/lstm_wyniki_known_pred.pkl')
df_lat_lstm = pd.read_pickle('results/lstm_wyniki_only_labels.pkl')

In [48]:
df_kn_gru = pd.read_pickle('results/gru_wyniki_known_pred.pkl')
df_lat_gru = pd.read_pickle('results/gru_wyniki_only_labels.pkl')

df_kn_transformer = pd.read_pickle('results/transformer_wyniki_known_pred.pkl')
df_lat_transformer = pd.read_pickle('results/transformer_wyniki_only_labels.pkl')

In [53]:
print('Best hiperparameters for LSTM model to predict unknown labels:')
df_kn_lstm[df_kn_lstm.val_accuracy_max == df_kn_lstm.val_accuracy_max.max()]

Best hiperparameters for LSTM model to predict unknown labels:


,lstm_units,dropout_rate,learning_rate,epoch,batch,loss_max,accuracy_max,val_loss_max,val_accuracy_max
20,128.0,0.1,0.001,20.0,128.0,0.032814,0.988296,1.067564,0.789644


In [54]:
print('Best hiperparameters for LSTM model to predict known labels:')
df_lat_lstm[df_lat_lstm.val_accuracy_max == df_lat_lstm.val_accuracy_max.max()]

Best hiperparameters for LSTM model to predict known labels:


,lstm_units,dropout_rate,learning_rate,epoch,batch,loss_max,accuracy_max,val_loss_max,val_accuracy_max
27,128.0,0.2,0.001,30.0,32.0,0.182577,0.944293,1.821681,0.611564


In [56]:
print('Best hiperparameters for GRU model to predict unknown labels:')
df_kn_gru[df_kn_gru.val_accuracy_max == df_kn_gru.val_accuracy_max.max()]

Best hiperparameters for GRU model to predict unknown labels:


,gru_units,dropout_rate,learning_rate,epoch,batch,loss_max,accuracy_max,val_loss_max,val_accuracy_max
18,128.0,0.1,0.001,20.0,32.0,0.126454,0.953914,0.341508,0.85584


In [57]:
print('Best hiperparameters for GRU model to predict known labels:')
df_lat_gru[df_lat_gru.val_accuracy_max == df_lat_gru.val_accuracy_max.max()]

Best hiperparameters for GRU model to predict known labels:


,gru_units,dropout_rate,learning_rate,epoch,batch,loss_max,accuracy_max,val_loss_max,val_accuracy_max
28,128.0,0.2,0.001,30.0,64.0,0.481092,0.837943,0.764989,0.751649


In [58]:
print('Best hiperparameters for Transformer model to predict unknown labels:')
df_kn_transformer[df_kn_transformer.val_accuracy_max == df_kn_transformer.val_accuracy_max.max()]

Best hiperparameters for Transformer model to predict unknown labels:


,num_heads,dropout_rate,learning_rate,epoch,batch,num_layer,loss_max,accuracy_max,val_loss_max,val_accuracy_max
25,4.0,0.1,0.001,20.0,128.0,2.0,0.190858,0.922019,0.457584,0.834951


In [59]:
print('Best hiperparameters for Transformer model to predict known labels:')
df_lat_transformer[df_lat_transformer.val_accuracy_max == df_lat_transformer.val_accuracy_max.max()]

Best hiperparameters for Transformer model to predict known labels:


,num_heads,dropout_rate,learning_rate,epoch,batch,num_layer,loss_max,accuracy_max,val_loss_max,val_accuracy_max
30,4.0,0.2,0.001,30.0,64.0,2.0,0.555792,0.808337,0.941868,0.717113


# Silence + Unknown + Label

In [2]:
import tensorflow as tf
model = tf.keras.models.load_model('models\\final\\FINAL_gru_known.h5')

In [39]:
# model.predict(test.batch(32))

In [11]:
import numpy as np
import models
from models import Lstm, Gru, Transformer, TransformerBlock
from dataset import LABELS
import pandas as pd

class final_model_SKL:
    def __init__(self, name, test, batch_size=32):

        model_known = tf.keras.models.load_model(f'models\\final\\FINAL_{name}_known.h5')
        model_label = tf.keras.models.load_model(f'models\\final\\FINAL_{name}_labels_only.h5')
        
        self.model_known = model_known
        self.model_label = model_label
        self.test = test
        self.batch_size = batch_size
        self.name = name

        LABELS_dict = {i:LABELS[i] for i in range(len(LABELS))}
        LABELS_dict[10] = 'unknown'
        self.LABELS_dict = LABELS_dict

        self.predicted_labels = None
        self.silence_labels = None

    def predict_silence(self):
        self.silence_labels = pd.read_pickle('silence_detected.pkl')
    
    def predict_label(self):
        self.predicted_labels = self.model_label.predict(self.test.batch(self.batch_size))

    def predict_if_known(self):
        self.predicted_labels_known = self.model_known.predict(self.test.batch(self.batch_size))

    def save_results(self, name):
        df = pd.read_csv('sample_submission.csv')
        x = pd.concat([pd.DataFrame(self.predicted_labels), pd.DataFrame(self.silence_labels)], axis=1)
        x.columns = ['label', 'silence']
        res = np.where(x['silence'] == 0 , 'silence', x['label'].apply(lambda x: self.LABELS_dict[x]))
        df['label'] = res
        df.to_csv(f'results\\submission_{name}.csv', index=False, header=True)

In [12]:
d = final_model_SKL('lstm', test, batch_size=32)

In [15]:
d.predict_label()
d.predict_silence()
d.predict_if_known()

4955/4955 [==============================] - 484s 97ms/step


In [34]:
df = pd.read_csv('sample_submission.csv')
x = pd.concat([pd.DataFrame(np.argmax(d.predicted_labels, axis=1)),pd.DataFrame(d.silence_labels), pd.DataFrame(np.argmax(d.predicted_labels_known, axis=1))], axis=1)
x.columns = ['label', 'silence','unknown']
# unknown == 1 => unknown
res = np.where(x['silence'] == 0 , 'silence', np.where((x['unknown'] == 1) , 'unknown', 'dupa'))
df['label'] = res
name = 'lstm'
df.to_csv(f'results/FINAL_submission_SKL_{name}.csv', index=False, header=True)

In [40]:
from dataset import unknown_detection_validation
x = d.model_known.predict(unknown_detection_validation.batch(32))

213/213 [==============================] - 27s 106ms/step


In [46]:
pd.DataFrame(np.argmax(x,axis=1)).value_counts()

1    4203
0    2595
dtype: int64

In [36]:
x.unknown.value_counts()

1    93924
0    64614
Name: unknown, dtype: int64

# Gru

In [37]:
d = final_model_SKL('gru', test, batch_size=32)

OSError: No file or directory found at models\final\FINAL_gru_known.h5

In [4]:
model = Gru(from_path = 'models\\gru_mod.h5')
model2 = Transformer(from_path = 'models\\transformer_mod.h5')
d = final_model(model, test, test_silence)

In [24]:
d.predict_silence()

In [ ]:
d.predict_label()

In [32]:
print('Number of silences:', len(d.test) - d.silence_labels.sum())
print('Total number of observations:', len(d.test))

Number of silences: 801
Total number of observations: 158538


In [ ]:
d.save_results()

In [33]:

class ensamble_model:
    def __init__(self):
        # train = pd.read_pickle('features_test_silence.pkl')
        # y_train = np.array([x[1] for x in train])
        # labels = list(np.unique(y_train))
        # self.labels_silence = labels
        # self.silence_det = TensorflowDataset('features_test_silence.pkl', labels=self.labels_silence).dataset
        
        train = pd.read_pickle('features_test.pkl')
        y_train = np.array([x[1] for x in train])
        labels = list(np.unique(y_train))
        self.labels_label = labels
        self.label_det = TensorflowDataset('features_test.pkl', labels=self.labels_label).dataset
    
        self.silence_model = tf.keras.models.load_model('models\\silence_model.h5')
        self.final_model = 'dupa'#tf.keras.models.load_model('models\\final_model.h5')
    
    def predict_silence(self):
        pred = self.silence_model.predict(self.silence_det)


    def save():
        # otwórz plik csv do zapisu
        with open('predictions.csv', mode='w', newline='') as file:
            # utwórz obiekt writer i ustaw separator na przecinek
            writer = csv.writer(file, delimiter=',')
            
            # zapisz nagłówek
            writer.writerow(['col1', 'col2', 'col3', 'col4', 'prediction'])
            
            # zapisz dane i przewidywania
            for i in range(len(data)):
                row = data[i] + [predictions[i]]
                writer.writerow(row)

